# 親クラスと子クラス間で情報をやり取りする
by Gemini

属性を直接変更する代わりに、メソッド呼び出しを通じて親クラスと子クラス間で情報をやり取りする設計は、より明確で保守性の高いコードにつながることがあります。

この方法では、子クラスが親クラスに「依頼」する形で状態を変更したり、親クラスが子クラスに「通知」する形で処理を促したりします。属性の直接的な操作を隠蔽することで、クラス間の依存関係をよりコントロールしやすくなります。

以下に、具体的な方法と例をいくつかご紹介します。

## 2. 方法 1: 子クラスから親クラスのメソッドを呼び出して状態を変更する:
子クラスが、親クラスに用意された専用のメソッドを呼び出すことで、親クラスの属性を変更します。
- 子クラスに主導権

In [ ]:
class Parent:
    def __init__(self, data):
        # この属性を直接変更しないでくださいね
        self._data = data

    def get_data(self):
        return self._data

    def update_data(self, new_data):
        print(f"Parent: Updating data to {new_data}")
        self._data = new_data

    def process(self):
        print(f"Parent processing with data: {self._data}")


class Child(Parent):
    def __init__(self, child_data, parent_instance):
        super().__init__(parent_instance.get_data())
        # 親のデータを初期化時に取得
        # Parent クラスの _data を引き継ぎ、独自に持つことにより、親の状態を変更しない
        # ただし、親のメソッドを通じて変更することは可能

        # 親クラスのインスタンスを保持
        self.parent = parent_instance

        # 子クラス独自のデータ
        self.child_specific_data = child_data

    def request_parent_data_update(self, new_data):
        # 親クラスの update_data メソッドを呼び出すことで、親の _data 属性を変更
        print("Child: Requesting parent to update data...")
        self.parent.update_data(new_data)

        # self.parent._data = new_data  # 直接変更は可能。しかし避けるべき
        # 親クラスのメソッドを通じて状態を変更することが推奨される


# インスタンスの作成
parent_obj = Parent("初期データ")
child_obj = Child("子データ", parent_obj)

# 子クラスから親クラスのメソッドを通じて状態を変更
child_obj.request_parent_data_update("メソッド経由で変更されたデータ")

# 親クラスのインスタンスで変更された属性を確認し、処理を継続
parent_obj.process()

Child クラスの request_parent_data_update メソッドが、親クラス Parent の update_data メソッドを呼び出すことで、親の _data 属性を変更しています。

子クラスは親の内部構造（_data という属性名）を直接知る必要がありません。

### 3. 類似の別案。デメリットも教えるよ
親クラスのインスタンスと子クラスのインスタンスが、互いに相手のインスタンスへの参照を持ち、その参照を通じて属性にアクセスし、値を読み書きします。

In [ ]:
class Parent:
    def __init__(self, data):
        self.data = data

        # 子クラスのインスタンスへの参照を保持する場所
        self.child = None

    def process(self):
        print(f"Parent processing with its own data: {self.data}")
        # 子クラスのインスタンスがある場合
        if self.child:
            # 子のデータを参照したり
            print(f"Parent also sees child's data: {self.child.child_specific_data}")

            # 子に親のデータを渡したり
            self.data = self.child.modify_parent_data(self.data)  # 子に親のデータを変更させる
            print(f"Parent's data after child modification: {self.data}")


class Child(Parent):
    def __init__(self, parent_instance, child_data):
        super().__init__(parent_instance.data)  # 親のデータを初期化時にコピー
        self.child_specific_data = child_data

        # 親子双方向の参照を持つ

        # 子から見た親
        self.parent = parent_instance

        # 親から見た子
        # 親に子インスタンスへの参照を渡す。元はNoneだった所に自分をセット
        self.parent.child = self

    def modify_parent_data(self, parent_data):
        # 親のデータを変更する
        self.parent.data = self.child_specific_data + " (modified by child)"
        return self.parent.data


# インスタンスの作成
parent_obj = Parent("初期データ")
child_obj = Child(parent_obj, "子固有のデータ")

# 処理の実行
parent_obj.process()  # 初期データで処理

print("\n--- 子クラスによる親データの変更 ---")
child_obj.modify_parent_data()

print("\n--- 変更後の親クラスのデータで処理 ---")
parent_obj.process()

print("\nParent's data:", parent_obj.data)
print("Child's specific data:", child_obj.child_specific_data)

#### 4. デメリット
密結合：
- このような双方向の参照は、親クラスと子クラスが強く結びつくことを意味します。一方のクラスの変更が他方に大きな影響を与える可能性があり、疎結合な設計に比べて柔軟性が低くなることがあります。

循環参照とメモリ管理: 
- 複雑な参照関係は、ガーベジコレクションの効率に影響を与える可能性も考慮する必要があります（Python のガーベジコレクタは循環参照も適切に処理することが多いですが、念頭に置いておくべきです）。

設計の意図: 
- なぜ親と子が互いに直接の参照を持ち、属性を読み書きする必要があるのか、その設計意図を明確にすることが重要です。より明確なインターフェース（メソッド呼び出し）を通じて情報のやり取りを行う方が、長期的に見て保守しやすい場合があります。

#### 4. 疎結合を目指すなら抽象基底クラス

抽象基底クラスは、特定のインターフェースを強制することで、
親クラスが期待するメソッドを子クラスが必ず実装するように保証し、
ポリモーフィズムを通じて柔軟な振る舞いを実現することを主な目的としています。

親子の直接的な情報のやり取りというよりは、役割と責任の分離、共通インターフェースの定義に重点が置かれます。

## 2. 方法 2: 親クラスが子クラスのメソッドを呼び出して情報を取得したり、処理を促したりする (コールバック):
親クラスが、子クラスに実装された特定のメソッドを呼び出すことで、子クラスの状態を取得したり、子クラスに何らかの処理を実行させたりします。
- 親クラスに主導権

別の言い方をすると、
抽象基底クラス ABC を利用し、
親クラスと子クラス間の役割と責任を明確化
- 親クラスは、委任したい処理を抽象メソッドとして表現。実装は子クラスにまかせる
- 子クラスは、処理を実装する。任された処理を確実に行う責任を追う
- 抽象基底クラスは、親クラスから子クラスに委任するインターフェース（抽象メソッド）を決める



In [ ]:
from abc import ABC, abstractmethod

class Parent:
    def __init__(self, data: str):
        self._data = data

    def process(self, processor: "DataTransformer"):
        print(f"Parent processing with initial data: {self._data}")
        
        # データが None や空文字でないことを確認
        if not self._data:
            print("Error: Data is empty or None, processing aborted.")
            return
        
        updated_data = processor.transform_data(self._data)
        self._data = updated_data
        print(f"Parent processing with transformed data: {self._data}")

class DataTransformer(ABC):
    @abstractmethod
    def transform_data(self, data: str) -> str:
        pass

class ChildProcessor(DataTransformer):
    def transform_data(self, data: str) -> str:
        print("ChildProcessor: Transforming data...")
        return data.upper() if data else "INVALID DATA"

# インスタンスの作成
parent_obj = Parent("lowercase data")
child_processor = ChildProcessor()

# 親クラスのメソッドに子クラスのインスタンスを渡して処理を依頼
parent_obj.process(child_processor)


親プロセスで自分の属性（self._data）について処理（processメソッド）を行う時、引数に（処理を任せたい）子クラスのインスタンスを伴う。

実際の処理は、その子クラスに定義されたメソッド（transform_data）が行う。

注意：
- 子クラスは親クラスの継承ではなく、class DataTransformer(ABC) の継承だ。


### 3. 抽象基底クラス ABC 
抽象基底クラス
- インターフェースを決める
  - メソッド名 = 抽象メソッド
  - 引数やその型

子クラスは、抽象基底クラスDataTransformerを継承
- 抽象基底クラスのインターフェースを実装（transform_data メソッド）

親クラスは、子クラスで実装されたインターフェースを用いて自分の属性を処理する




### 3. 抽象基底クラスのメリット
ここでは、親クラスが特定の処理を子クラスに委譲したり、子クラスの状態に応じて振る舞いを変更したりするために、子クラスに定義されたメソッドを呼び出します。これは、親クラスが直接子クラスの具体的な実装を知らなくても、共通のインターフェースを通じて連携できるという点で強力です。

#### 4. インターフェースの強制:

ABC を使うと、親クラスが期待するメソッド（コールバックメソッド）を抽象メソッドとして定義できます。

子クラスは、ABC を継承する際に、これらの抽象メソッドを必ず実装しなければなりません。もし実装を怠ると、子クラスのインスタンスを生成しようとした時点で TypeError が発生します。
これにより、親クラスは、連携する子クラスが必ず必要なメソッドを持っていることを保証できます。インターフェースの不一致による実行時のエラーを防ぎ、より堅牢なコードになります。

#### 4. 設計の意図の明確化:

ABC は、親クラスと子クラス間の役割と責任を明確に示します。

親クラスは「こういう処理を子クラスに委ねる」という意図を抽象メソッドとして表現し、子クラスは「親クラスから要求されるこの処理を実装する」という責任を負うことがコード上で明示されます。
これにより、コードの可読性と理解度が向上し、設計の意図が伝わりやすくなります。

#### 4. ポリモーフィズムの実現:

ABC を継承した複数の異なる子クラスは、同じ抽象メソッドをそれぞれ独自の方法で実装できます。

親クラスは、これらの子クラスのインスタンスを同じインターフェース（抽象メソッド）を通じて扱うことができるため、多様な振る舞いを統一的に扱えるポリモーフィズムを実現できます。
親クラスのコードを変更することなく、新しい振る舞いを持つ子クラスを簡単に追加できます。

#### 4. 型の安全性と静的解析の向上:

ABC を使うことで、型ヒントを使って親クラスが期待するコールバックメソッドの型を明示的に記述できます。

型チェッカー（mypy など）は、子クラスがこのインターフェースを正しく実装しているかどうかを静的にチェックできるため、型に関連するエラーをコンパイル時に発見しやすくなります。

#### 4. まとめ
抽象基底クラスをコールバックの仕組みに用いることで、インターフェースの強制、設計意図の明確化、ポリモーフィズムの実現、型の安全性と静的解析の向上といった多くのメリットが得られ、より柔軟で堅牢なオブジェクト指向設計が可能になります。親クラスと子クラス間の連携をより安全かつ明確に行いたい場合に、抽象基底クラスは非常に有効なツールとなります。

## 2. 方法 3: プロパティ (@property, @setter) を使用して制御されたアクセスを提供する:
親クラスが、属性へのアクセスや変更を制御するためのプロパティを提供します。子クラスは、これらのプロパティを通じて間接的に属性を操作します。

親クラスが属性を持ち、子クラスがそれを変更する

class Parent:
    def __init__(self, data):
        self._data = data

    @property
    def data(self):
        print("Parent: Getting data...")
        return self._data

    @data.setter
    def data(self, new_data):
        print(f"Parent: Setting data to {new_data}")
        self._data = new_data

    def process(self):
        print(f"Parent processing with data: {self.data}")

class Child(Parent):
    def __init__(self, data):
        super().__init__(data)  # 親のコンストラクタを呼び出す

    def update_parent_data(self, new_data):
        print("Child: Updating parent's data via property...")
        self.data = new_data  # 親の setter が呼ばれる

# インスタンスの作成
parent_obj = Parent("初期データ")
child_obj = Child("子データ")

# 子クラスからプロパティを通じて状態を変更
child_obj.update_parent_data("プロパティ経由で変更されたデータ")

# 親クラスのインスタンスで変更された属性を確認し、処理を継続
parent_obj.process()


Parent クラスが data プロパティを提供しており、Child クラスは self.data = ... という形で親の setter を呼び出して状態を変更しています。これにより、親クラスは属性の変更を監視したり、バリデーションなどの追加のロジックを挟んだりすることができます。


### 3. メリット:

- 明確なインターフェース: 
  - クラス間のやり取りがメソッド呼び出しという明確な形で行われるため、コードの意図が理解しやすくなります。
- カプセル化の強化: 
  - 親クラスの内部構造（属性名や実装の詳細）を隠蔽し、子クラスからの不適切なアクセスを防ぐことができます。
- 柔軟性の向上: 
  - 状態の変更時に追加のロジック（バリデーション、通知、ログ記録など）を組み込みやすくなります。
- 保守性の向上: 
  - クラス間の依存関係が明確になるため、一方のクラスの変更が他方に与える影響を把握しやすくなります。

属性の直接的な変更が簡便である一方、
より複雑なシステムや長期的な保守を考えると、メソッド呼び出しを通じて情報をやり取りする設計を検討する価値があります。


# Copilotにも抽象基底クラス（ABC）を使った方がよいと言われた
Copilot

抽象基底クラス（ABC） を使うことで密結合を解消し、設計の柔軟性と拡張性を向上できます。特に、親と子が互いに依存しすぎる関係をインターフェース経由の間接的なやり取りに置き換えることで、クラス同士の独立性を高めることができます。

## 2. ABCを使うべき理由
親子間の強すぎる結びつきをなくす
- 直接 self.parent や self.child を持つ代わりに、抽象メソッドを介して必要な処理を統一的に行える。

拡張性の向上
- 親クラスや子クラスを自由に追加・変更できる。

テストのしやすさ
- 依存性を減らし、単体テストが容易になる。

In [ ]:
from abc import ABC, abstractmethod

class ParentBase(ABC):
    """親クラスのインターフェース（抽象基底クラス）"""
    @abstractmethod
    def process(self):
        pass

    @abstractmethod
    def update_data(self, new_data: str):
        pass

class Parent(ParentBase):
    """具象クラスの親"""
    def __init__(self, data: str):
        self.data = data

    def process(self):
        """親独自の処理
        """
        print(f"Parent processing with its own data: {self.data}")

    def update_data(self, new_data: str):
        """親の属性を変更するだけ。新しい値は他者に用意してもらう＝委任
        """
        print(f"Parent: Updating data to {new_data}")
        self.data = new_data

class DataModifier(ABC):
    """データを加工する抽象基底クラス"""
    @abstractmethod
    def modify_data(self, parent_data: str) -> str:
        pass

class Child(DataModifier):
    """親と密結合しない形でデータを加工する子クラス

    子ならではの処理をこっちに記述
    """
    def __init__(self, child_data: str):
        """子固有の属性を初期化

        親の属性は持ち込まない
        子は、親への直接的な参照を持たない
        親のデータは、メソッドの引数としてのみ受け付けます
        """
        self.child_specific_data = child_data

    def modify_data(self, parent_data: str) -> str:
        """データ加工処理
        
        引数は親のデータかもしれないがあまり気にしない
        直接親の属性を参照しない
        子はデータの加工にのみ注力する
        """
        print("Child: Modifying parent's data...")
        return parent_data + " (modified by child)"

# インスタンスの作成
parent_obj = Parent("初期データ")
child_modifier = Child("子固有のデータ")

# データの更新
new_parent_data = child_modifier.modify_data(parent_obj.data)
parent_obj.update_data(new_parent_data)

# 処理の実行
parent_obj.process()


## 2. ポイント
### 3. 親クラスと子クラスを完全に分離

ParentBase を使って、親クラスのインターフェースを定義。

Parent クラスは ParentBase を実装し、親の動作をカプセル化。

子クラスは Parent に直接依存せず、インターフェース DataModifier を通じてデータ処理のみを担当。

### 3. 子クラスは親クラスのインスタンスを持たない

Child は modify_data() を通じてデータ加工を行うが、Parent への直接的な参照を持たない。

### 3. 親はインターフェースを通じてデータを更新

Child は親を持たず、親のデータを引数として受け取り加工する。

### 3. 親と子で別々の抽象基底クラスを用意
もし「ABCを継承するクラスを減らしたい」「もっとシンプルにしたい」ということであれば、ParentBase をなくして Parent を直接使う形にもできます。
ただ、抽象基底クラスを使うことで、複数の親・複数のデータ加工クラスが共存できる拡張性が生まれる

#### 4. ParentBase（親の抽象基底クラス）

ParentBase は親クラスのインターフェースとして働きます。

これを継承する Parent は、具体的な処理 (process や update_data) を実装します。

もし Parent とは異なる別の親クラスを作りたくなった場合、ParentBase を継承する新しいクラスを追加できるので柔軟性が増します。

#### 4. DataModifier（データを加工する抽象基底クラス）

DataModifier はデータを加工するためのインターフェースとして働きます。

Child は DataModifier を継承し、modify_data の具体的な処理を定義します。

これにより、異なるデータ変換ロジックを持つクラス（例えば AdvancedModifier や SpecialModifier）を追加できるので、拡張がしやすくなります。

# 抽象基底クラスのブログその1
https://qiita.com/TrashBoxx/items/7a76e46122191529c526 Pythonの抽象クラス(ABCmeta)を詳しく説明したい


## 2. 基本の書き方 0

In [ ]:
# 抽象クラスの定義
from abc import ABC, abstractmethod

class Person0(ABC):

    @abstractmethod
    def greeting(self):
        pass


In [ ]:
# 継承したクラス
class Yamada0(Person):

    def greeting(self):
        print("こんにちは、山田です。")

## 2. 抽象クラス 応用

### 3. classmethod、staticmethod、propertyの抽象化

#### 4. 抽象クラス Person

In [ ]:
class Person(ABC):

    @staticmethod
    @abstractmethod
    def age_fudging(age):
        """サバを読んで年齢を返す
        """
        pass

    @classmethod
    @abstractmethod
    def weight_fudging(cls, weight):
        """サバを読んで体重を返す
        """
        pass

    @property
    @abstractmethod
    def age(self):
        """年齢プロパティ
        """
        pass

    # ageプロパティのセッターのインターフェース
    @age.setter
    @abstractmethod
    def age(self, val):
        """年齢セッター
        """
        pass

    @property
    @abstractmethod
    def weight(self):
        """体重プロパティ
        """
        pass

    # weightプロパティのセッターのインターフェース
    @weight.setter
    @abstractmethod
    def weight(self, val):
        """体重セッター
        """
        pass


#### 4. 実装 山田
本当の年齢・体重は、外部から触ってほしくないので、
変数名の先頭に __ を付けて名前マングリングを施す。

外部から年齢・体重を読み書きするため、
getter/setter（プロパティ）を用意する。

In [ ]:
class Yamada(Person):

    def __init__(self):
        # 本当の年齢
        self.__age = 30

        # 本当の体重
        self.__weight = 120
        # インスタンスの変数名の先頭に _ を使うと、慣習的に非公開
        # インスタンスの変数名の先頭に __ を使うのは、名前マングリングと言われ、外部から直接アクセス不能
    
    #10歳サバ読み
    @staticmethod
    def age_fudging(age):
        return age - 10
    
    #20kgサバ読み
    @classmethod
    def weight_fudging(cls, weight):
        return weight - 20
    
    @property
    def age(self):
        """
        ageプロパティが呼ばれたら、本当の年齢より10歳サバ読みするメソッドを介してから年齢を返す
        """
        return Yamada.age_fudging(self.__age)
    
    @age.setter
    def age(self):
        return
    
    @property
    def weight(self):
        return self.weight_fudging(self.__weight)
    
    @weight.setter
    def weight(self):
        return


使用例

In [ ]:
y = Yamada()

# インスタンスの age, weight プロパティにアクセスすると、サバを読んだ結果を返す
print(y.age, y.weight)
# 20 100

# 抽象基底クラスのブログその2
https://note.com/leapcell/n/n9fe4c1076f3a
Python応用：抽象基底クラスABCを使いこなす

## 2. 初期：シンプルだが厳密さに欠ける
クラス継承は使っているが。

問題：
- サブクラスに必要なすべてのメソッドを実装するよう強制することができません。
- 基底クラスのメソッドのシグネチャ（パラメータリスト）がサブクラスのそれと一致しない場合があります。
- 明確なインターフェイス契約がありません。


In [ ]:
class LeapCellFileHandler:
    def read(self, filename):
        pass

    def write(self, filename, data):
        pass

class LeapCellJsonHandler(LeapCellFileHandler):
    def read(self, filename):
        import json
        with open(filename, 'r') as f:
            return json.load(f)

    def write(self, filename, data):
        import json
        with open(filename, 'w') as f:
            json.dump(data, f)

class LeapCellCsvHandler(LeapCellFileHandler):
    def read(self, filename):
        import csv
        with open(filename, 'r') as f:
            return list(csv.reader(f))


## 2. 改良バージョン：抽象基底クラスを使用

abstractmethodデコレータを使って抽象メソッドをマーク


In [ ]:
from abc import ABC, abstractmethod

class LeapCellFileHandler(ABC):
    @abstractmethod
    def read(self, filename: str):
        """ファイルの内容を読み取る"""
        pass

    @abstractmethod
    def write(self, filename: str, data: any):
        """ファイルに内容を書き込む"""
        pass

class LeapCellJsonHandler(LeapCellFileHandler):
    def read(self, filename: str):
        import json
        with open(filename, 'r') as f:
            return json.load(f)

    def write(self, filename: str, data: any):
        import json
        with open(filename, 'w') as f:
            json.dump(data, f)


使い方

In [ ]:
# インスタンス作成
# handler = LeapCellJsonHandler(...)

## 2. さらなる最適化：型ヒントとインターフェイス制約の追加

In [ ]:
from abc import ABC, abstractmethod
from typing import Any, List, Dict, Union

class LeapCellFileHandler(ABC):
    @abstractmethod
    def read(self, filename: str) -> Union[Dict, List]:
        """ファイルの内容を読み取り、解析されたデータ構造を返す"""
        pass

    @abstractmethod
    def write(self, filename: str, data: Union[Dict, List]) -> None:
        """データ構造をファイルに書き込む

        Union[Dict, List] は、どちらの型でも許容するの意
        """
        pass

    @property
    @abstractmethod
    def supported_extensions(self) -> List[str]:
        """サポートされるファイル拡張子のリストを返す
        
        抽象メソッドではなく、抽象プロパティであることを指定するデコレータの付け方
        """
        pass

class LeapCellJsonHandler(LeapCellFileHandler):
    def read(self, filename: str) -> Dict:
        import json
        with open(filename, 'r') as f:
            return json.load(f)

    def write(self, filename: str, data: Dict) -> None:
        import json
        with open(filename, 'w') as f:
            json.dump(data, f)

    @property
    def supported_extensions(self) -> List[str]:
        return ['.json']

# 使用例
def process_leapcell_file(handler: LeapCellFileHandler, filename: str) -> None:
    """
    Args:
        handler: どの実装クラスが来ても良い
    """
    if any(filename.endswith(ext) for ext in handler.supported_extensions):
        # 抽象基底クラスとは関係ないが、戸惑ったので調べた
        # 分解:
        # handler.supported_extensionsについて。プロパティは、.プロパティ名 にて値を返す
        # any([ ]) とリスト内包表記でリストにしてっしまっても同じ判定結果となるが、いったんリストを完成させる点が残念な点。
        # any( ) であれば、ジェネレータ式を一回ずつ評価し、Trueが出た時点で判定結果が確定するので、無駄なループを回さない

        # 実装メソッド readとwriteを呼ぶ
        data = handler.read(filename)
        handler.write(f'processed_{filename}', data)
    else:
        raise ValueError(f"{filename}に対してサポートされていないファイル拡張子です")


### 3. 抽象プロパティ
上に出て来たけど、こう書いてね

抽象基底クラスでの書き方:
```
    @property
    @abstractmethod
    def プロパティ名
        pass
```

実装クラスでの書き方:
```
    @property
    def プロパティ名
```

実行する場所での書き方:
```
インスタンス = 実装クラス()
インスタンス.プロパティ名
```


## 2. raise NotImplementedErrorを使う例
抽象基底クラスの中に、通常のインスタンスメソッドを定義したなら、
継承先クラスのインスタンスでそのメソッドを使用できる。

In [ ]:
from abc import ABC, abstractmethod

class LeapCellFileHandler(ABC):

    # 抽象メソッド
    @abstractmethod
    def read(self, filename: str) -> Dict:
        """サブクラスで必ずオーバーライド（再実装）する必要がある
        """
        pass

    # 通常のインスタンスメソッド
    def process(self, filename: str) -> Dict:
        data = self.read(filename)
        if not self._validate(data):
            raise ValueError("データ形式が無効です")
        return self._transform(data)

    # 通常のインスタンスメソッド
    def _validate(self, data: Dict) -> bool:
        raise NotImplementedError("サブクラスは実装する必要があります")
        # 解説: 要するに、抽象メソッドではないから実装は必須ではないけれど例外発生させたくないサブクラスでは実装してねという意味。
        #       だが、素直に抽象メソッドにした方が設計意図が明確になる

    # 通常のインスタンスメソッド
    def _transform(self, data: Dict) -> Dict:
        # デフォルトの実装
        return data


last

---